In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
import re
import sys
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import RFECV

In [2]:
def extract_int(x):
    int_list = re.findall(r'\d+', x)
    return int("".join(int_list))


In [3]:
df = pd.read_csv('loan-data\Anonymize_Loan_Default_data.csv', encoding= 'ISO-8859-1')

In [4]:
df.describe(include = 'all')

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,...,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,repay_fail
count,38480.000000,3.848000e+04,3.848000e+04,38479.000000,38479.000000,38479.000000,38480,38480.00000,38479.000000,37487,...,38479.000000,38479.000000,38479.000000,38479.000000,38479.000000,38409,38479.000000,3383,38477,38480.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,11,...,NaN,NaN,NaN,NaN,NaN,105,NaN,103,110,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,36 months,NaN,NaN,10+ years,...,NaN,NaN,NaN,NaN,NaN,Jun-16,NaN,Jul-16,Jun-16,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,28593,NaN,NaN,8465,...,NaN,NaN,NaN,NaN,NaN,1022,NaN,909,11342,NaN
mean,19240.500000,6.649979e+05,8.261899e+05,11094.727644,10831.856337,10150.141518,NaN,12.16430,323.163255,NaN,...,22.108501,11980.696892,11274.519569,9646.412705,2232.768235,NaN,2614.441757,NaN,NaN,0.151481
std,11108.363516,2.192322e+05,2.793531e+05,7405.416042,7146.853682,7128.026828,NaN,3.73744,209.089097,NaN,...,11.588602,9006.505205,8946.229941,7051.828302,2570.177312,NaN,4391.969583,NaN,NaN,0.358522
min,1.000000,1.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,NaN,0.00000,0.000000,NaN,...,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.000000
25%,9620.750000,4.983645e+05,6.384620e+05,5200.000000,5100.000000,4950.000000,NaN,9.62000,165.740000,NaN,...,13.000000,5463.099238,4811.735000,4400.000000,657.700000,NaN,212.010000,NaN,NaN,0.000000
50%,19240.500000,6.443195e+05,8.242545e+05,9750.000000,9600.000000,8495.792749,NaN,11.99000,277.980000,NaN,...,20.000000,9673.221341,8953.240000,8000.000000,1335.090000,NaN,526.000000,NaN,NaN,0.000000
75%,28860.250000,8.265608e+05,1.034706e+06,15000.000000,15000.000000,14000.000000,NaN,14.72000,429.350000,NaN,...,29.000000,16402.394995,15486.925000,13315.100000,2795.020000,NaN,3169.815000,NaN,NaN,0.000000


#### Going to want to drop the items that will clearly have no correlation or will be far too difficult to evaluate effectively. (e.g. zip codes are a discrete variable with far too many values to be an effective predictor)

In [5]:
to_drop = ['Unnamed: 0', 'id', 'member_id', 'zip_code']

#Also want to determine if any of the dates have information that could be used to determine default

null_count = df.isnull().sum(axis = 0)
print(null_count.sort_values(ascending=False)[0:5])
#Although null values for months since last delinquency likely imply the loanee has never been delinquent, it is a quite dangerous assumption to make.
#Simply assuming all 24363 values 
to_drop.extend(['mths_since_last_delinq','next_pymnt_d'])



bins = [0,3,9,10]
labels = ['0 - 2 years', '3 - 9 years', '10+ years']
#Have to drop na before performing this operation
df.dropna(subset=['emp_length'], inplace=True)
df['emp_length']  = df['emp_length'].apply(extract_int)
print(df['emp_length'])
df['emp_length'] = pd.cut(df['emp_length'], bins=bins, labels=labels)
df['emp_length'].value_counts()


next_pymnt_d              35097
mths_since_last_delinq    24363
emp_length                  993
last_pymnt_d                 71
revol_util                   59
dtype: int64
0         1
1         4
2         4
3        10
4        10
         ..
38475     3
38476     9
38477    10
38478     1
38479     7
Name: emp_length, Length: 37487, dtype: int64


0 - 2 years    16050
3 - 9 years    12972
10+ years       8465
Name: emp_length, dtype: int64

In [6]:
#Also going to break up some salary info
bins = [0,40000,60000,80000,sys.maxsize]
labels = ['0 - 40k', '40k - 60k', '60k - 80k', '80k+']

df['annual_inc'] = pd.cut(df['annual_inc'], bins=bins, labels=labels)
df['annual_inc']

0              NaN
1          0 - 40k
2        40k - 60k
3        40k - 60k
4          0 - 40k
           ...    
38475    60k - 80k
38476    60k - 80k
38477         80k+
38478    40k - 60k
38479      0 - 40k
Name: annual_inc, Length: 37487, dtype: category
Categories (4, object): ['0 - 40k' < '40k - 60k' < '60k - 80k' < '80k+']

In [7]:
df.drop(to_drop, axis=1)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,...,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,repay_fail
0,0.0,0.0,0.00000,36 months,0.00,0.00,0 - 2 years,RENT,NaN,Not Verified,...,0.00%,1.0,0.000000,0.00,0.00,0.00,Jan-07,0.00,Jan-07,1
1,2500.0,2500.0,2500.00000,36 months,13.98,85.42,3 - 9 years,RENT,0 - 40k,Not Verified,...,21.30%,10.0,3075.291779,3075.29,2500.00,575.29,Jul-13,90.85,Jun-16,0
2,5000.0,5000.0,5000.00000,36 months,15.95,175.67,3 - 9 years,RENT,40k - 60k,Not Verified,...,99.90%,15.0,2948.760000,2948.76,1909.02,873.81,Nov-11,175.67,Mar-12,1
3,7000.0,7000.0,7000.00000,36 months,9.91,225.58,10+ years,MORTGAGE,40k - 60k,Not Verified,...,47.20%,20.0,8082.391880,8082.39,7000.00,1082.39,Mar-14,1550.27,Mar-14,0
4,2000.0,2000.0,2000.00000,36 months,5.42,60.32,10+ years,RENT,0 - 40k,Not Verified,...,0%,15.0,2161.663244,2161.66,2000.00,161.66,Feb-14,53.12,Jun-16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38475,3000.0,3000.0,3000.00000,36 months,11.99,99.63,0 - 2 years,RENT,60k - 80k,Verified,...,73.50%,22.0,3586.619764,3586.62,3000.00,586.62,Aug-14,100.30,Jun-16,0
38476,10400.0,10400.0,10400.00000,36 months,13.49,352.88,3 - 9 years,RENT,60k - 80k,Source Verified,...,39.70%,33.0,12703.534030,12703.53,10400.00,2303.53,Sep-14,393.08,Jun-16,0
38477,16000.0,10550.0,10531.35818,60 months,14.96,250.77,10+ years,MORTGAGE,80k+,Source Verified,...,62.20%,25.0,14202.267530,14163.31,10550.00,3652.27,Jun-13,5439.96,Feb-16,0
38478,10000.0,10000.0,10000.00000,36 months,16.89,355.99,0 - 2 years,RENT,40k - 60k,Not Verified,...,53.30%,12.0,12815.178320,12815.18,10000.00,2815.18,Aug-14,380.63,Sep-15,0


In [8]:
df.columns

Index(['Unnamed: 0', 'id', 'member_id', 'loan_amnt', 'funded_amnt',
       'funded_amnt_inv', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'purpose', 'zip_code', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths',
       'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'repay_fail'],
      dtype='object')